In [9]:
import pandas as pd
import numpy as np

In [41]:
df= pd.read_csv(r'letras.txt', sep='\t', skiprows=0, header=None)
df.head(12)

,0,1,2,3,4
0,1.0,1.0,1.0,1.0,1.0
1,1.0,-1.0,-1.0,-1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,-1.0,-1.0,-1.0,1.0
4,1.0,-1.0,-1.0,-1.0,1.0
5,NaN,NaN,NaN,NaN,NaN
6,1.0,1.0,1.0,1.0,1.0
7,1.0,-1.0,-1.0,-1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,-1.0,-1.0,-1.0,1.0


Paso las matrices a vectores 

In [102]:
data= []
toAdd= []
for index, row in df.iterrows():
    for column in df:
        number= row[column]
        if (not pd.isna(number)):
            toAdd.append(int(number))
        else:
            data.append(toAdd)
            toAdd=[]
            break

print("L: ", data[11])
print("P: ", data[15])
print("T: ", data[19])
print("X: ", data[23])

L:  [1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]
P:  [1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1]
T:  [1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1]
X:  [1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1]


Analizo la ortogonalidad de los vectores (más cercano a 0 significa más ortogonales)

In [68]:
print("L, X: ", np.dot(dfToRows[11], dfToRows[23]))
print("L, T: ", np.dot(dfToRows[11], dfToRows[19]))
print("L, P: ", np.dot(dfToRows[11], dfToRows[15]))

print("X, T: ", np.dot(dfToRows[23], dfToRows[19]))
print("X, P: ", np.dot(dfToRows[23], dfToRows[15]))

print("T, P: ", np.dot(dfToRows[19], dfToRows[15]))

L, X:  1
L, T:  -3
L, P:  -1
X, T:  1
X, P:  -5
T, P:  3


In [186]:
class Hopfield:
   
    # init method or constructor 
    def __init__(self, data):
        self.W= self.calculateSynapticWeight(data)
        
    def calculateSynapticWeight(self, data):
        firstRow= data.pop(0)
        Kt= np.array([firstRow])
        while len(data) > 0:
            newRow= np.array(data.pop(0))
            Kt= np.concatenate([Kt, [newRow]])
        K= Kt.T
        
        N= len(firstRow)
        w= (1/N) * np.dot(K, Kt)
        w= w - np.identity(N) * w[0][0]
        
        return w
   
    # Sample Method 
    def run(self, S):
        i= 0
        prevS= S
        actS= self.prodW(prevS)
        while (not np.array_equal(prevS, actS) and i < 5000000):
            prevS= actS;
            actS= self.prodW(prevS)
            i= i+1
        print(i)
        return(actS)
    
    def printW(self):
        print(self.W)
        
    def prodW(self, S):
        return np.sign(np.dot(self.W, S))

In [188]:
toHop= [data[11], data[15], data[19], data[23]]
h= Hopfield(toHop)

In [189]:
print(data[11])
a= h.run(data[11])
print(a)

[1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]
0
[ 1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1.
 -1. -1.  1.  1.  1.  1.  1.]


In [94]:
L = np.array([1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1])
P= np.array([1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1])
print(L)
print(P)

[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
  1]
[ 1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1
 -1]


In [114]:
matrix= np.array([L])
print(matrix)
print()
matrix= np.concatenate([matrix, [P]])
print(matrix)

[[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
   1]]

[[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
   1]
 [ 1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1
  -1]]


In [112]:
toSend= [data[11], data[15], data[19], data[23]]
print(toSend[0])

[1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]


In [117]:
toSend= [data[11], data[15], data[19], data[23]]
Kt= np.array([toSend.pop(0)])
while len(toSend) > 0:
    newRow= np.array(toSend.pop(0))
    Kt= np.concatenate([Kt, [newRow]])
K= Kt.T

print(toSend)
print(Kt)
print(K)

[]
[[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
   1]
 [ 1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1
  -1]
 [ 1  1  1  1  1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1
  -1]
 [ 1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1
   1]]
[[ 1  1  1  1]
 [-1  1  1 -1]
 [-1  1  1 -1]
 [-1  1  1 -1]
 [-1  1  1  1]
 [ 1  1 -1 -1]
 [-1 -1 -1  1]
 [-1 -1  1 -1]
 [-1 -1 -1  1]
 [-1  1 -1 -1]
 [ 1  1 -1 -1]
 [-1  1 -1 -1]
 [-1  1  1  1]
 [-1  1 -1 -1]
 [-1  1 -1 -1]
 [ 1  1 -1 -1]
 [-1 -1 -1  1]
 [-1 -1  1 -1]
 [-1 -1 -1  1]
 [-1 -1 -1 -1]
 [ 1  1 -1  1]
 [ 1 -1 -1 -1]
 [ 1 -1  1 -1]
 [ 1 -1 -1 -1]
 [ 1 -1 -1  1]]


In [120]:
toSend= [data[11], data[15], data[19], data[23]]
print(len(toSend.pop(0)))

25
